# Blue Brain BioExplorer

![](./images/bioexplorer_banner.png)

In [59]:
from bioexplorer import BioExplorer, Protein, Surfactant, Membrane, Volume, Virus,\
                        Cell, Sugars, Vector2, Vector3, Quaternion
import nglview

be = BioExplorer('localhost:5000')

In [60]:
status = be.reset()

### Resources

In [61]:
resource_folder = '../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'
glycan_folder = pdb_folder + 'glycans/'

glucose_path = pdb_folder + 'glucose.pdb'
lactoferrin_path=pdb_folder + 'immune/1b0l.pdb'
defensin_path = pdb_folder + 'immune/1ijv.pdb'

complex_paths = [glycan_folder + 'complex/33.pdb', glycan_folder + 'complex/34.pdb',
                 glycan_folder + 'complex/35.pdb',glycan_folder + 'complex/36.pdb']
high_mannose_paths = [glycan_folder + 'high-mannose/1.pdb', 
                      glycan_folder + 'high-mannose/2.pdb',
                      glycan_folder + 'high-mannose/3.pdb',
                      glycan_folder + 'high-mannose/4.pdb']
hybrid_paths = [glycan_folder + 'hybrid/24.pdb']
o_glycan_paths = [glycan_folder + 'o-glycan/12.pdb']

### Configuration

In [62]:
# Scene
scene_size = 250

# Proteins
protein_radius_multiplier = 1.0
protein_representation = be.REPRESENTATION_ATOMS_AND_STICKS
glycan_radius_multiplier = 1.0
glycan_representation = be.REPRESENTATION_ATOMS_AND_STICKS
protein_load_hydrogen = False

# Viruses
nb_protein_s = 62
nb_protein_e = 42
nb_protein_m = 50
add_rna = False
add_glycans = True

## Camera

In [75]:
status = be.core_api().set_camera(
    orientation=[0.0394, 0.359, -0.112, 0.925],
    position=[119.168, -68.452, 97.192],
    target=[90.192, -61.656, 64.614]    
)

## Scene description

### Coronavirus

In [64]:
def add_virus(name, position, open_conformation_indices=list()):
    
    closed_conformation_indices = list()
    for i in range(nb_protein_s):
        if i not in open_conformation_indices:
            closed_conformation_indices.append(i)
            
#     closed_conformation_indices = [10]

    virus_protein_s = Protein(
        sources=[
            pdb_folder + '6vyb.pdb',  # Open conformation
            pdb_folder + 'sars-cov-2-v1.pdb'  # Closed conformation
        ],
        load_hydrogen=protein_load_hydrogen, occurences=nb_protein_s, 
        assembly_params=Vector2(11.5, 0.0), cutoff_angle=0.999,
        orientation=Quaternion(0.0, 1.0, 0.0, 0.0),
        instance_indices=[open_conformation_indices, closed_conformation_indices])

    virus_protein_m = Protein(
        sources=[pdb_folder + 'QHD43419a.pdb'], load_hydrogen=protein_load_hydrogen,
        occurences=nb_protein_m, assembly_params=Vector2(2.0, 0.0),
        cutoff_angle=0.999, orientation=Quaternion(0.99, 0.0, 0.0, 0.135))

    virus_protein_e = Protein(
        sources=[pdb_folder + 'QHD43418a.pdb'], load_hydrogen=protein_load_hydrogen, 
        occurences=nb_protein_e, assembly_params=Vector2(3.0, 0.0), 
        cutoff_angle=0.9999, orientation=Quaternion(0.705, 0.705, -0.04, -0.04))

    virus_membrane = Membrane(
        sources=[pdb_folder + 'membrane/popc.pdb'],
        occurences=15000)

    rna_sequence = None
    clip_planes = list()
    if add_rna:
        clip_planes.append([0, 0, -1, 15])
        import math
        rna_sequence = RNASequence(
            source=rna_folder + 'sars-cov-2.rna',
            assembly_params=Vector2(11.0, 0.5),
            t_range=Vector2(0, 30.5 * math.pi), shape=be.RNA_SHAPE_TREFOIL_KNOT,
            shape_params=Vector3(1.51, 1.12, 1.93))

    coronavirus = Virus(
        name=name, protein_s=virus_protein_s, protein_e=virus_protein_e,
        protein_m=virus_protein_m, membrane=virus_membrane, rna_sequence=rna_sequence,
        assembly_params=Vector2(45.0, 1.5))

    be.add_virus(
        virus=coronavirus, position=position, representation=protein_representation,
        atom_radius_multiplier=protein_radius_multiplier, clipping_planes=clip_planes)

    if add_glycans:
        # High-mannose
        indices_closed = [61, 122, 234, 603, 709, 717, 801, 1074]
        indices_open = [61, 122, 234, 709, 717, 801, 1074]
        be.add_multiple_glycans(
            assembly_name=name, glycan_type=be.NAME_GLYCAN_HIGH_MANNOSE, 
            protein_name=be.NAME_PROTEIN_S_CLOSED, paths=high_mannose_paths, 
            indices=indices_closed,
            representation=glycan_representation,
            atom_radius_multiplier=glycan_radius_multiplier
        )
        be.add_multiple_glycans(
            assembly_name=name, glycan_type=be.NAME_GLYCAN_HIGH_MANNOSE, 
            protein_name=be.NAME_PROTEIN_S_OPEN, paths=high_mannose_paths,
            indices=indices_open,
            representation=glycan_representation,
            atom_radius_multiplier=glycan_radius_multiplier
        )

        # Complex
        indices_closed = [17, 74, 149, 165, 282, 331,
                          343, 616, 657, 1098, 1134, 1158, 1173, 1194]
        indices_open = [17, 74, 149, 165, 282, 331, 343, 657, 1098, 1134, 1158, 1173, 1194]
        be.add_multiple_glycans(
            orientation=Quaternion(0.0, 0.0, 0.0, 1.0),
            assembly_name=name, glycan_type=be.NAME_GLYCAN_COMPLEX, 
            protein_name=be.NAME_PROTEIN_S_CLOSED, paths=complex_paths, 
            indices=indices_closed,
            representation=glycan_representation,
            atom_radius_multiplier=glycan_radius_multiplier
        )
        be.add_multiple_glycans(
            orientation=Quaternion(0.0, 0.0, 0.0, 1.0),
            assembly_name=name, glycan_type=be.NAME_GLYCAN_COMPLEX, 
            protein_name=be.NAME_PROTEIN_S_OPEN, paths=complex_paths, 
            indices=indices_open, representation=glycan_representation,
            atom_radius_multiplier=glycan_radius_multiplier
        )

        # O-Glycans
        for index in [323, 325]:
            o_glycan_name = name + '_' + be.NAME_GLYCAN_O_GLYCAN + '_' + str(index)
            o_glycan = Sugars(
                assembly_name=name, name=o_glycan_name, source=o_glycan_paths[0],
                protein_name=name + '_' + be.NAME_PROTEIN_S_CLOSED, 
                site_indices=[index],
                representation=glycan_representation,
                atom_radius_multiplier=glycan_radius_multiplier
            )
            be.add_sugars(o_glycan)


        # High-mannose glycans on Protein M
        indices = [5]
        high_mannose_glycans = Sugars(
            orientation=Quaternion(0.707, 0.0, 0.0, 0.707),
            assembly_name=name, name=be.NAME_GLYCAN_HIGH_MANNOSE, 
            protein_name=name + '_' + be.NAME_PROTEIN_M, source=high_mannose_paths[0], 
            site_indices=indices,
            representation=glycan_representation,
            atom_radius_multiplier=glycan_radius_multiplier
        )
        status = be.add_glycans(high_mannose_glycans)


        # Complex glycans on Protein E
        indices = [48, 66]
        complex_glycans = Sugars(
            orientation=Quaternion(0.707, 0.0, 0.0, 0.707),
            assembly_name=name, name=be.NAME_GLYCAN_COMPLEX, 
            protein_name=name + '_' + be.NAME_PROTEIN_E, source=complex_paths[0], 
            site_indices=indices,
            representation=glycan_representation,
            atom_radius_multiplier=glycan_radius_multiplier
        )
        status = be.add_glycans(complex_glycans)

In [65]:
add_virus(name='Coronavirus', position=Vector3(75, 0.0, 60.0), 
          open_conformation_indices=[1])

### Host cell

In [66]:
# ACE2 receptor definition
ace2_receptor = Protein(
    sources=[pdb_folder + '6m18.pdb'],
    occurences=1,
    position=Vector3(0.0, 6.0, 0.0),
)

# Membrane definition
membrane_size = scene_size
membrane_height = 80
membrane = Membrane(
    sources=[pdb_folder + 'membrane/popc.pdb'],
    occurences=30000
)

# Cell definition
name='Cell'
cell = Cell(
    name=name,
    size=Vector2(membrane_size, membrane_height),
    shape=be.ASSEMBLY_SHAPE_SINUSOIDAL,
    membrane=membrane, receptor=ace2_receptor)

# Add cell to scene
status = be.add_cell(
    cell=cell, representation=protein_representation,
    atom_radius_multiplier=protein_radius_multiplier,
    random_seed=1,
    position=Vector3(-7, -82.5, -3.0)
)

status = be.set_protein_color_scheme(
    name, name + '_' + be.NAME_RECEPTOR, 
    be.COLOR_SCHEME_CHAINS, 'OrRd_r', 7)

In [67]:
be.add_multiple_glycans(
    orientation=Quaternion(0.707, 0.0, 0.0, 0.707),
    representation=glycan_representation, assembly_name=name, 
    glycan_type=be.NAME_GLYCAN_COMPLEX,
    protein_name=be.NAME_RECEPTOR, paths=complex_paths, 
    indices=[53, 90, 103, 322, 432, 690])

be.add_multiple_glycans(
    orientation=Quaternion(0.707, 0.0, 0.0, 0.707),
    representation=glycan_representation, assembly_name=name,
    glycan_type=be.NAME_GLYCAN_HYBRID,
    protein_name=be.NAME_RECEPTOR, paths=hybrid_paths, 
    indices=[546])

indices = [[155, Quaternion(0.0, 0.707, 0.0, 0.707)],
           [730, Quaternion(0.0, 0.707, 0.0, 0.707)]]
for index in indices:
    o_glycan_name = name + '_' + be.NAME_GLYCAN_O_GLYCAN + '_' + str(index[0])
    o_glycan = Sugars(
        assembly_name=name, name=o_glycan_name, 
        source=o_glycan_paths[0],
        protein_name=name + '_' + be.NAME_RECEPTOR, 
        representation=glycan_representation,
        chain_ids=[2, 4], site_indices=[index[0]], 
        orientation=index[1])
    be.add_sugars(o_glycan)

## Materials

In [68]:
be.apply_default_color_scheme(
    shading_mode=be.SHADING_MODE_PERLIN,
    user_parameter=0.03, specular_exponent=5, glossiness=0.1)

IntProgress(value=0, max=30)

## Functional regions

### Spike

In [69]:
# RGB color palette for amino acid indices
grey = [0.2, 0.2, 0.2]
dark_green = [0.0, 1.0, 0.0]
light_green = [0.0, 1.0, 0.0]
red = [1.0, 0.0, 0.0]
green = [0.0, 1.0, 0.0]
blue = [0.0, 0.0, 1.0]
yellow = [1.0, 1.0, 0.0]
cyan = [1.0, 0.0, 1.0]

region_indices_and_colors = [
    [   1, grey ], [   16, grey], [  306, grey], [  330, grey], [438, dark_green], 
    [ 507, green], [  522, grey], [  816, grey ], [  835, grey ], [908, grey],
    [ 986, grey ], [ 1076, grey], [ 1274, grey], [ 2000, grey ]
]

# Build a palette according to region colors
palette = list()
for index in range(len(region_indices_and_colors)-1):
    for i in range(region_indices_and_colors[index + 1][0] - 
                   region_indices_and_colors[index][0]):
        palette.append(region_indices_and_colors[index][1])

# Apply palette to other chains
status = be.set_protein_color_scheme(
    assembly_name='Coronavirus', name='Coronavirus_' + be.NAME_PROTEIN_S_OPEN, 
    color_scheme=be.COLOR_SCHEME_CHAINS,
    palette_name='Greys_r', palette_size=100)

# Apply palette to region color scheme, only for chain #2
status = be.set_protein_color_scheme(
    assembly_name='Coronavirus', name='Coronavirus_' + be.NAME_PROTEIN_S_OPEN, 
    color_scheme=be.COLOR_SCHEME_REGION, palette=palette,
    chain_ids=[2]
)

### Receptor

In [70]:
# RGB color palette for amino acid indices
grey = [0.5, 0.5, 0.5]
dark_blue = [0.0, 0.0, 0.5]
light_blue = [0.0, 0.0, 1.0]

region_indices_and_colors = [
    [   1, grey      ], [   5, grey],  [  30, light_blue],  [  41, dark_blue],
    [  82, light_blue], [  84, grey],  [ 353, grey      ],  [ 357, grey     ],
    [ 800, grey      ]]

# Build a palette according to region colors
palette = list()
for index in range(len(region_indices_and_colors)-1):
    for i in range(region_indices_and_colors[index + 1][0] - 
                   region_indices_and_colors[index][0]):
        palette.append(region_indices_and_colors[index][1])

# Apply palette to other chains
name = 'Cell'
status = be.set_protein_color_scheme(
    assembly_name=name, name=name + '_' + be.NAME_RECEPTOR, 
    color_scheme=be.COLOR_SCHEME_CHAINS,
    palette_name='Greys_r', palette_size=100)

# Apply palette to region color scheme, only for chain #2
status = be.set_protein_color_scheme(
    assembly_name=name, name=name + '_' + be.NAME_RECEPTOR, 
    color_scheme=be.COLOR_SCHEME_REGION, palette=palette,
    chain_ids=[4])

## Rendering settings

In [ ]:
status = be.core_api().set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer',
    samples_per_pixel=1, subsampling=4, max_accum_frames=64)
params = be.core_api().BioExplorerRendererParams()
params.gi_samples = 3
params.gi_weight = 0.4
params.gi_distance = 10
params.shadows = 0.8
params.soft_shadows = 1
params.fog_start = 1500
params.fog_thickness = 1500
params.max_bounces = 1
status = be.core_api().set_renderer_params(params)

## Snapshots

In [71]:
output_folder = '/tmp/'

In [76]:
from braynsmediamaker import MovieMaker
mm = MovieMaker(be.core_api())
mm.create_snapshot(
    path=output_folder + 'protein_s_and_ace2_receptor.png',
    size=[2048,2048], samples_per_pixel=64)

IntProgress(value=0, description='In progress...')

In [77]:
mm.create_snapshot(
    path=output_folder + 'protein_s_and_ace2_receptor_art_v3.png',
    size=[3840,2160], samples_per_pixel=64)

IntProgress(value=0, description='In progress...')